In [1]:
from AOC_utils import get_day
import numpy as np

day = 21
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 21 input already downloaded
5


['480A', '143A', '983A', '382A', '974A']

In [2]:
example = '''
029A
980A
179A
456A
379A
'''.split('\n')[1:-1]

In [3]:
from itertools import permutations

In [4]:
def solve_day(data):
    part1 = 0
    part2 = 0

    final_codes = data

    dir_map = {'<': np.array([0, -1]), '>': np.array([0, 1]), '^': np.array([-1, 0]), 'v': np.array([1, 0])}

    numpad = np.array([['7', '8', '9'], 
                       ['4', '5', '6'], 
                       ['1', '2', '3'],
                       [' ', '0', 'A']])
    
    robot_pad = np.array([[' ', '^', 'A'],
                         ['<', 'v', '>']])
    
    def sim_pad(start_loc, direction):
        new_loc = start_loc.copy()
        new_loc += dir_map[direction]
        return numpad[tuple(new_loc)]
    
    numpad_mapping = {}
    for current_n in numpad.flatten():
        for target_n in numpad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            current_loc = np.argwhere(numpad == current_n)[0]
            target_loc = np.argwhere(numpad == target_n)[0]

            distance = current_loc - target_loc
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            # moves = np.unique([''.join(x) for x in list(permutations(list(move)))], axis=0)
            moves = [ # always going to be better to do repeated moves than alternate
                (left * '<') + (right * '>') + (up * '^') + (down * 'v'), 
                 (up * '^') + (down * 'v') + (left * '<') + (right * '>')
            ]

            if moves[0] == '':
                numpad_mapping[current_n + target_n] = ['']
                continue

            good_moves = []
            for move in moves:
                loc = current_loc.copy()
                valid = True
                for direction in move:
                    if sim_pad(loc, direction) == ' ':
                        valid = False
                        break
                    loc += dir_map[direction]
                if valid:
                    good_moves.append(move)

            numpad_mapping[current_n + target_n] = good_moves

    robot_mapping = {}
    for current_n in robot_pad.flatten():
        for target_n in robot_pad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            distance = np.argwhere(robot_pad == current_n)[0] - np.argwhere(robot_pad == target_n)[0]
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            move = (up * '^') + (down * 'v') + (left * '<') + (right * '>')

            if move == '':
                robot_mapping[current_n + target_n] = 'A'
                continue

            moves = np.unique([''.join(x) for x in list(permutations(list(move)))], axis=0).tolist()

            good_moves = []
            for move in moves:
                loc = np.argwhere(robot_pad == current_n)[0]
                valid = True
                for direction in move:
                    if sim_pad(loc, direction) == ' ':
                        valid = False
                        break
                    loc += dir_map[direction]
                if valid:
                    good_moves.append(move)
            moves = sorted(good_moves, key=lambda x: len(x))

            robot_mapping[current_n + target_n] = moves[0] + 'A'

    def fcode_to_robot(c, current='A'):
        return [x + 'A' for x in numpad_mapping[current + c]]
    
    def robot_to_robot(code, current='A'):
        moves_so_far = ''
        for c in code:
            moves_so_far += robot_mapping[current + c]
            current = c
        return moves_so_far
    
    def process_codelist(final_codes, depth):
        total = 0
        for code in final_codes:
            # print(code)
            current = 'A'
            for ch in code:
                code_possabilities = fcode_to_robot(ch, current)

                for d in range(depth):
                    # print(d)
                    code_possabilities = [robot_to_robot(r) for r in code_possabilities]

                code_possabilities = sorted(code_possabilities, key=lambda x: len(x))
                total += len(code_possabilities[0]) * int(code[:-1])
                current = ch
        return total

    part1 = process_codelist(final_codes, 2)
    print("part 1:", part1)

    # part2 = process_codelist(final_codes, 25)
    # print("part 2:", part2)

solve_day(example)

part 1: 126384


In [5]:
solve_day(input_data)

part 1: 206798


In [6]:
206798

206798